In [1]:
! pip install konlpy

In [2]:
import json
import os
import nltk
from konlpy.tag import Okt

In [3]:
def open_txt(filename):
    with open(filename, 'r',encoding = 'utf-8') as f:  
        data = [line.split('\t') for line in f.read().splitlines()]
        data = data[1:]
    return data

In [7]:
train_data = open_txt('/content/train_sample.txt')
test_data = open_txt('/content/test_sample.txt')

In [8]:
okt = Okt()
okt.pos('ㅠㅠ 영화관에서 잠든 유일한 영화..')

[('ㅠㅠ', 'KoreanParticle'),
 ('영화관', 'Noun'),
 ('에서', 'Josa'),
 ('잠든', 'Verb'),
 ('유일한', 'Adjective'),
 ('영화', 'Noun'),
 ('..', 'Punctuation')]

In [9]:
# 하나의 문장을 토큰화 한 후 텍스트와 품사태깅을 / 구분자로 묶어준다.

def tokenizing(docs):
    return ['/'.join(t) for t in okt.pos(docs, norm=True, stem=True)]

In [10]:
train_list = []  
test_list = []  
for i in train_data:
    try:
        train_value = [tokenizing(i[1]), i[2]]
        train_list.append(train_value)
    except:
        pass

for i in test_data:
    try:
        test_value = [tokenizing(i[1]), i[2]]
        test_list.append(test_value)
    except:
        pass

In [11]:
tokens = [t for d in train_list for t in d[0]]
text = nltk.Text(tokens,name='NMSC')
text.vocab().most_common(10) #vocab().most_common(10) - 텍스트 빈도 상위 10개 보여주기
selected_words = [f[0] for f in text.vocab().most_common(10000)]

In [12]:
def term_frequency(doc):
    return [doc.count(word) for word in selected_words]

train_x = [term_frequency(d) for d, _ in train_list]

In [13]:
# 1번째리뷰를 상위 10000개와 각각 매칭하여 각 10000개의 단어가 해당 문장에 얼마나 포함되는지를 확인

test_x = [term_frequency(d) for d, _ in test_list]

train_y = [c for _, c in train_list]
test_y = [c for _, c in test_list]


In [14]:
import numpy as np

x_train = np.asarray(train_x).astype('float32')
x_test = np.asarray(test_x).astype('float32')

y_train = np.asarray(train_y).astype('float32')
y_test = np.asarray(test_y).astype('float32')


In [16]:
from keras import losses
from keras import metrics
from keras import models
from keras import layers
from tensorflow.keras import optimizers

model = models.Sequential()
model.add(layers.Dense(64, activation='relu', input_shape=(10000,))) 
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

# 모델 생성
model.compile(optimizer=optimizers.RMSprop(lr=0.001),
              loss=losses.binary_crossentropy,
              metrics=[metrics.binary_accuracy])

# 모델 학습
model.fit(x_train, y_train, epochs=10, batch_size=512)
results = model.evaluate(x_test, y_test)

# 예측 결과
results

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


Epoch 1/10
79/79 [==============================] - 8s 47ms/step - loss: 0.4724 - binary_accuracy: 0.7986
Epoch 2/10
79/79 [==============================] - 4s 46ms/step - loss: 0.3397 - binary_accuracy: 0.8577
Epoch 3/10
79/79 [==============================] - 4s 45ms/step - loss: 0.2985 - binary_accuracy: 0.8753
Epoch 4/10
79/79 [==============================] - 4s 46ms/step - loss: 0.2655 - binary_accuracy: 0.8910
Epoch 5/10
79/79 [==============================] - 7s 84ms/step - loss: 0.2344 - binary_accuracy: 0.9054
Epoch 6/10
79/79 [==============================] - 4s 45ms/step - loss: 0.2053 - binary_accuracy: 0.9184
Epoch 7/10
79/79 [==============================] - 4s 45ms/step - loss: 0.1772 - binary_accuracy: 0.9316
Epoch 8/10
79/79 [==============================] - 4s 45ms/step - loss: 0.1514 - binary_accuracy: 0.9431
Epoch 9/10
79/79 [==============================] - 4s 44ms/step - loss: 0.1281 - binary_accuracy: 0.9539
Epoch 10/10
313/313 [=========================

[0.6580227017402649, 0.8220000267028809]

In [18]:
import pandas as pd
df = pd.read_csv('/content/df_notk.csv', encoding = 'cp949')

In [19]:
df

,Unnamed: 0,id,title,review,score,sentiment
0,0,17998769,반지의 제왕: 왕의 귀환,판타지를 넘어 영화 역사에 남을 명작이다. 내가 이걸 왜 극장에서 못봤을까.... ...,10.0,1
1,1,17998767,나일 강의 죽음,이집트여행하는느낌의 영화,8.0,1
2,2,17998766,킹메이커,목포 연설 장면은 넋을 잃고 보게 된다.,10.0,1
3,3,17998765,기억의 밤,그저 그렇네요. 뻔한 반전과 결말.,4.0,0
4,4,17998764,더 배트맨,더 배트맨 조커처럼 몰입감이 있으면 추천 반대로 지루하면 비추천,1.0,0
...,...,...,...,...,...,...
9325,9352,17988421,오! 수정,이해할 수 없는 남자와 이해해주는 여자,7.0,0
9326,9353,17988420,이상한 나라의 수학자,배우들 연기력이 장난아님. 역시 최민식 배우는 클라스가 다름!,10.0,1
9327,9354,17988419,블랙라이트,그래도 최근에 나왔던 영화들에 비해 내용이 되게 흥미진진했음 리암 니슨이 덜뛰는 대...,10.0,1
9328,9355,17988418,더 배트맨,누구나 시작은 어설픈 법. 그래서 더 매력적인 히어로,10.0,1


In [20]:
def input_text(text):
    token = tokenizing(text)
    tf =term_frequency(token)

    data = np.expand_dims(np.asarray(tf).astype('float32'), axis=0)

    score = float(model.predict(data)) #새로운 데이터를 받으면 결과 예측
    if(score > 0.5):
        print("{} : 긍정 [{:.2f}%] \n".format(text, score * 100))
    else:
        print("{} : 부정 [{:.2f}%] \n".format(text, (1 - score) * 100))

In [21]:
for i in range(len(df)) :
    input_text(df['review'][i])

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
일본식 멜로영화의 정수 : 부정 [90.23%] 

뽕쟁이 아들을 독재국가로 부터 지키기 위한 아버지의 눈물나는 똥꼬쑈. 감동실화 : 부정 [99.99%] 

한마디로 표현 하자면 개꿀잼 : 긍정 [99.11%] 

연출도 별로고 내용도 그닥이고 개연성도 심하게 없고진짜 보고난 뒤 머리가 멍해지는 영화는 처음이네 : 부정 [100.00%] 

가볍게 보기좋은 미야자키 하야오 스타일 애니메이션 : 긍정 [99.69%] 

우엑…왜 이딴 영화를 만든거 : 부정 [99.99%] 

나를 위해 나를 빠뜨려야 하는 처절함과 애처로움 : 긍정 [100.00%] 

최민식 배우님 연기도 일품이였지만, 어찌보면 뻔한 결과임에도 불구하고 수학이라는 신선한 소재와 잔잔한 여운 및 스토리가 정말 좋았습니다.수학에 조금씩 관심을 가지고 있는 학생이나 현재 수학을 공부하는 학생이라면 더욱더 흥미를 가지고 수학에 흥미가 생길 수도 있을 것 같아요.영화관에서 마블이나 히어로물 아닌 영화 보고 재미있게 잘봤다 생각한 영화는 오랜만이였습니다! : 긍정 [100.00%] 

아름다운 배경과 음악, 현실과 사랑의 마지막결과아름다운영화네요 : 긍정 [99.99%] 

. : 부정 [64.90%] 

잘만든 영화역시 감독님 : 긍정 [99.01%] 

아역 너무귀여움♥♥♥ : 부정 [76.25%] 

훌륭한 재난영화 : 긍정 [99.09%] 

두시간 이십오분이 하루만큼  길게 느껴집니다. : 긍정 [53.04%] 

세련된 연출, 명배우들의 연기로 찢었다. 추천! : 긍정 [98.66%] 

최동훈의 스타일이 확립된 데뷔작 : 긍정 [99.62%] 

개성적인 캐릭터들을 보는 재미 : 긍정 [96.81%] 

존엄을 위해서 싸운다는 말이 참 기억에 남습니다, 그리고 마지막 독일군 장교가 선의를 배풀어 주었을때 여러가지 감정이 북받쳐 주인공이 울었던 장면이 참 슬펐습니다. : 긍정 [99.97%] 

진짜 평점 댓글이 있긴한가보다. 